In [1]:
import os
from datetime import datetime
import logging

import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
from simpletransformers.classification import ClassificationModel, ClassificationArgs
#from simpletransformers.t5 import T5Model, T5Args
import warnings
import pandas as pd
import os
from datetime import datetime
import logging
import glob
from pathlib import Path
import csv

In [2]:
import spacy
from spacy.language import Language
from spacy import displacy
import time

@Language.component("newsent")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        #print(token.text, token.text in ("’s", "'s"))
        if token.text.upper() in (";", "--", "\n\n", "\n", "QUARTERLY", "STORY", "\n\n\n\n", "\n\n\n"):
            #print("Detected:", token.text)
            doc[token.i].is_sent_start = True
    return doc

#spacy.require_gpu()
nlp = spacy.load("../../Summary/NER/RelateEntity/train/model-best-local")
nlp.add_pipe('sentencizer')
nlp.add_pipe('newsent', name="newsent", last=True)

c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\users\ankan\appdata\local\programs\python\python37\lib\site-packages\spacy_transformers\pipeline_component.py:406: UserWarning: Automatically converting a transformer component from spacy-transformers v1.0 to v1.1+. If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spacy-transformers version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


<function __main__.set_custom_boundaries(doc)>

In [3]:
def getSentences(inputfile, nlp, text=None):
    if(not text):
        with open(inputfile, 'r', encoding="utf-8") as f:
            text = f.read()

    doc = nlp(text)
    sentences = [str(sent).strip() for sent in doc.sents]

    #print(len(sentences))
    return(sentences)

In [4]:
def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )
    return out_string

In [5]:
import hashlib
def dedupList(flist):
    hashList = list()
    newList = list()
    for item in flist:
        #print(item.upper())
        hash_object = hashlib.sha1(item.upper().encode("UTF-8"))
        hex_dig = hash_object.hexdigest()
        if hex_dig not in hashList:
            hashList.append(hex_dig)
            #print(item, hex_dig)
            newList.append(item)
    return(newList)

In [22]:
devDir = "../../Summary/DATA/PARAPHRASE/Dev"
trainDir = "../../Summary/DATA/PARAPHRASE/Train"
testDir = "../../Summary/DATA/PARAPHRASE/Test"

spdevDir = "../../Summary/DATA/SENTPAIR/Dev"
sptrainDir = "../../Summary/DATA/SENTPAIR/Train"
sptestDir = "../../Summary/DATA/SENTPAIR/Test"

rplStr = ["PG*** ", "ED*** ", "SCHQ*** ", "SCBQ*** ", "SCBF*** ", "SCHF*** ", "SCG*** ", "GF*** ", "GQ*** ", "SC*** ", "NOPAD*** "]

def preProcessSent(line):
    for s in rplStr:
        line = line.replace(s, "")
    if("TBLST***" in line or "TBLET***" in line or "CS***" in line or "@@@" in line or line == "\n" or line == "\n\n" or "https://finance.yahoo.com" in line):
        return(line, False)
    line = line.replace("\n", "")
    if(line == None or line == ""):
        return(line, False)
    line = line.replace("$(0.", "$(.")
    line = line.replace("($0.", "($.")
    line = line.replace("$0.", "$.")
    line = line.replace("$ 0.", "$.")
    return(line, True)

def createTrainingData(inputDir, ppModel=None, writetofile=False):
    files = glob.glob(inputDir+"/*_EP_YH.txt")
    #print(files)
    #print(inputDir)
    if(len(files) > 0):
        for file in (files):
            print("Input file " + file)
            basefile = os.path.basename(file)
            inputfile = os.path.splitext(basefile)[0]
            #print(inputfile)
            if not ppModel:
                outfilePath = inputfile + "_phrase.tsv"
                outfilePath = inputDir + "/" + outfilePath
                print("Phrase file " + outfilePath)
                outfile = Path(outfilePath)
                if outfile.is_file():
                    print("Phrase file " + str(outfile) + " already exists")
                    continue
                with open(outfile, "w", encoding = "utf-8") as of:
                    #cnt = 0
                    of.write("filename\tSentence1\tSentence2\n")
                    #with open(file, encoding="utf-8") as f:
                    sentences = getSentences(file, nlp)
                    #line = f.readline()
                    for line in sentences:
                        for s in rplStr:
                            line = line.replace(s, "")
                        if("TBLST***" in line or "TBLET***" in line or "CS***" in line or "@@@" in line or line == "\n" or line == "\n\n" or "https://finance.yahoo.com" in line):
                            #line = f.readline()
                            continue
                        line = line.replace("\n", "")
                        if(line == ""):
                            continue
                        #print(line)
                        #cnt = cnt + 1
                        phraseLine = line
                        of.write(inputfile+"\t"+line+"\t"+line+"\n")
                        #of.write(line + "\n")
                        #line = f.readline()
            else:
                f = None
                if(writetofile):
                    basefile = os.path.basename(file)
                    inputfile = os.path.splitext(basefile)[0]
                    print("Input file " + inputfile)
                    print(inputDir)
                    outfilePath = inputfile + "_sp.tsv"
                    outfilePath = inputDir + "/" + outfilePath
                    print("SP file " + outfilePath)
                    outfile = Path(outfilePath)
                    if outfile.is_file():
                        print("SP file " + str(outfile) + " already exists")
                        continue
                    f = open(outfile, "w", encoding = "utf-8")
                    f.write("filename\tSentence1\tSentence2\tTarget\n")
                sentences = getSentences(file, nlp)
                #line = f.readline()
                for line in sentences:
                    for s in rplStr:
                        line = line.replace(s, "")
                    if("TBLST***" in line or "TBLET***" in line or "CS***" in line or "@@@" in line or line == "\n" or line == "\n\n" or "https://finance.yahoo.com" in line):
                        #line = f.readline()
                        continue
                    line = line.replace("\n", "")
                    if(line == None or line == ""):
                        continue
                    #print(line)
                    #cnt = cnt + 1
                    #phraseLine = line
                    line = line.replace("$(0.", "$(.")
                    line = line.replace("($0.", "($.")
                    line = line.replace("$0.", "$.")
                    line = line.replace("$ 0.", "$.")
                    inp = [line]
                    #print(inp)
                    if(not writetofile):
                        print("ORIGINAL SENTENCE\n",line)
                        print("PREDICTED PARAPHRASE\n")
                        predicted = ppModel.predict(inp)
                        print(predicted)
                        print("\n")
                    else:
                        predicted = ppModel.predict(inp)
                        if(predicted):
                            predicted = dedupList(predicted[0])
                            #print("ORIGINAL SENTENCE\n",line)
                            #print("PREDICTED PARAPHRASE\n")
                            #print(predicted)
                            for pred in predicted:
                                if(f):
                                    f.write(inputfile+"\t"+line+"\t"+pred+"\t"+"1"+"\n")
                                    print(inputfile+"\t"+line+"\t"+pred+"\t"+"1")
                            #print("\n")
                if(f):
                    f.close()

In [29]:
def predictPhrase(line, ppModel, spModel, logf=None):
    nline, isProcess = preProcessSent(line)
    if not isProcess:
        return(nline)
    inp = [nline]
    print("ORIGINAL LINE:\n", line)
    if(logf):
        logf.write("ORIGINAL LINE:\n", line)
    predicted = ppModel.predict(inp)
    if(predicted):
        predicted = dedupList(predicted[0])
        print("PREDICTED PARAPHRASE:\n", predicted)
        if(logf):
            logf.write("PREDICTED PARAPHRASE:\n", predicted)
        for pred in predicted:
            spinp = [[nline, pred]]
            #print(spinp)
            spred, output = spModel.predict(spinp)
            print("PREDICTED SENTENCE PAIR:\n", spinp, spred)
            if(logf):
                logf.write("PREDICTED SENTENCE PAIR:\n", spinp, spred)
            if(spred[0] == 1):
                #print(spinp, spred)
                if(pred[0] == " "):
                    pred = pred[1:]
                print("PREDICTED SENTENCE:\n", pred)
                if(logf):
                    logf.write("PREDICTED SENTENCE:\n", pred)
                return(pred)
    print("PREDICTED SENTENCE:\n", nline)
    if(logf):
        logf.write("PREDICTED SENTENCE:\n", nline)
    return(nline)

In [8]:
devDataFile = "../../Summary/DATA/PARAPHRASE/Dev/dev.tsv"
trainDataFile = "../../Summary/DATA/PARAPHRASE/Train/train.tsv"

devSPFile = "../../Summary/DATA/SENTPAIR/Dev/dev.tsv"
trainSPFile = "../../Summary/DATA/SENTPAIR/Train/train.tsv"

def writeTrainingData(writeFile, writeDir):
    if("PARAPHRASE" in writeDir):
        files = glob.glob(writeDir+"/*_phrase.tsv")
    else:
        files = glob.glob(writeDir+"/*_sp.tsv")
    print(files)
    frames = list()

    if(len(files) > 0):
        for file in files:
            df = pd.read_csv(file, sep="\t", encoding = "utf-8").astype(str)
            df = df.dropna()
            df = df[df['Sentence1'].notna()]
            #print(df)
            frames.append(df)
    result = pd.concat(frames)
    print(result)
    result.to_csv(writeFile, sep='\t', index=False, header=True)

In [9]:
createTrainingData(devDir)
writeTrainingData(devDataFile, devDir)

Input file ../../Summary/DATA/PARAPHRASE/Dev\APPN_2023-02-16_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Dev/APPN_2023-02-16_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Dev\APPN_2023-02-16_EP_YH_phrase.tsv already exists
Input file ../../Summary/DATA/PARAPHRASE/Dev\BILL_2023-02-02_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Dev/BILL_2023-02-02_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Dev\BILL_2023-02-02_EP_YH_phrase.tsv already exists
Input file ../../Summary/DATA/PARAPHRASE/Dev\CFLT_2022-11-02_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Dev/CFLT_2022-11-02_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Dev\CFLT_2022-11-02_EP_YH_phrase.tsv already exists
Input file ../../Summary/DATA/PARAPHRASE/Dev\CRWD_2022-11-29_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Dev/CRWD_2022-11-29_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Dev\CRWD_2022-11-29_EP_YH_phrase.tsv already exists
Input file ../../Summary

In [10]:
createTrainingData(trainDir)
writeTrainingData(trainDataFile, trainDir)

Input file ../../Summary/DATA/PARAPHRASE/Train\APPN_2022-05-06_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Train/APPN_2022-05-06_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Train\APPN_2022-05-06_EP_YH_phrase.tsv already exists
Input file ../../Summary/DATA/PARAPHRASE/Train\APPN_2022-08-04_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Train/APPN_2022-08-04_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Train\APPN_2022-08-04_EP_YH_phrase.tsv already exists
Input file ../../Summary/DATA/PARAPHRASE/Train\APPN_2022-11-03_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Train/APPN_2022-11-03_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Train\APPN_2022-11-03_EP_YH_phrase.tsv already exists
Input file ../../Summary/DATA/PARAPHRASE/Train\APPN_2023-05-09_EP_YH.txt
Phrase file ../../Summary/DATA/PARAPHRASE/Train/APPN_2023-05-09_EP_YH_phrase.tsv
Phrase file ..\..\Summary\DATA\PARAPHRASE\Train\APPN_2023-05-09_EP_YH_phrase.tsv already exists


In [11]:
train_df = pd.read_csv(trainDataFile, sep="\t", encoding = "utf-8").astype(str)
eval_df = pd.read_csv(devDataFile, sep="\t", encoding = "utf-8").astype(str)

train_df = train_df.rename(
    columns={"Sentence1": "input_text", "Sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"Sentence1": "input_text", "Sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
train_df = train_df[["prefix", "input_text", "target_text"]]

eval_df["prefix"] = "paraphrase"
eval_df = eval_df[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
train_df = train_df[train_df['input_text'].notna()]

eval_df = eval_df.dropna()
eval_df = eval_df[eval_df['input_text'].notna()]

train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)
print("TRAIN DATA ..............")
print(train_df)

eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)
print("EVAL DATA ..............")
print(eval_df)

TRAIN DATA ..............
          prefix                                         input_text  \
0     paraphrase                                Appian Corporation.   
1     paraphrase   First quarter cloud subscription revenue incr...   
2     paraphrase         MCLEAN, Va., May 05, 2022 (GLOBE NEWSWIRE)   
3     paraphrase  -- Appian (Nasdaq: APPN) today announced finan...   
4     paraphrase   “We exceeded guidance and grew cloud subscrip...   
...          ...                                                ...   
1898  paraphrase  Accordingly, we are required to add back the n...   
1899  paraphrase  Additionally, we include the anti-dilutive imp...   
1900  paraphrase  We have not reconciled our expectations to non...   
1901  paraphrase  For those reasons, we are also unable to addre...   
1902  paraphrase  Accordingly, a reconciliation for the guidance...   

                                            target_text  
0                                   Appian Corporation.  
1    

In [9]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [10]:
def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum(
        [
            1 if label == pred else 0
            for label, pred in zip(labels, preds)
        ]
    )


In [14]:
model_args = Seq2SeqArgs()
#model_args = T5Args()
model_args.do_sample = True
model_args.train_batch_size = 4
model_args.use_multiprocessing = False
model_args.num_train_epochs = 2
#model_args.learning_rate = 5e-5
#model_args.no_save = True

model_args.eval_batch_size = 4
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.evaluate_during_training_steps = 50

model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.save_steps = -1

model_args.max_length = 64
#model_args.max_seq_length = 32
model_args.num_return_sequences = 10
model_args.top_k = 50
model_args.top_p = 0.95
model_args.fp16 = False
#model_args.num_beams = None

model_args.use_early_stopping = False
#model_args.early_stopping_delta = 0.01
#model_args.early_stopping_metric = "mcc"
#model_args.early_stopping_metric_minimize = False
#model_args.early_stopping_patience = 5
#model_args.evaluate_during_training_steps = 500
#model_args.wandb_project =  "visualization-demo"

model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-base",
    args=model_args,
    use_cuda=True,
)

#model = T5Model("t5", "t5-base", args=model_args, use_cuda=True)

# Train the model
model.train_model(
    train_df, eval_data=eval_df, matches=count_matches
)
results = model.eval_model(eval_df)

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/1505 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model: Training started


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/377 [00:00<?, ?it/s]

...

[' Appian Corporation.', ' Cloud subscription revenue is $65.8 million in fourth quarter.', 'Cloud subscription revenue is $236.9 million for full year.', ' MCLEAN, Va., Feb. 16, 2023 (GLOBE NEWSWIRE)', '-- Appian (Nasdaq: APPN) today announced financial results for the fourth quarter and full year ended December 31, 2022.', ' Fourth Quarter 2022 Financial Highlights:.', ' Cloud subscription revenue is $65.8 million.', 'Total subscriptions revenue, is $93.2 million.', 'Professional services revenue is $32.5 million.', 'Total revenue is $125.8 million.', 'Cloud subscription revenue retention rate is 115% as of December 31, 2022.', ' GAAP operating loss is $(40.6) million.', 'Non-GAAP operating loss is $(26.8) million.', ' GAAP net loss is $(34.4) million.', 'GAAP net loss per share is $(0.47).', 'Non-GAAP net loss is $(20.6) million.', 'Non-GAAP net loss per share is $(0.28).', 'Invalid Sentence.', 'Invalid Sentence.', 'We do not forecast foreign exchange rate movements.', ' Adjusted EB

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/627 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/157 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
model_args = Seq2SeqArgs()
#model_args = T5Args()
model_args.do_sample = True
#model_args.train_batch_size = 4
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.num_train_epochs = 2
#model_args.learning_rate = 5e-5
#model_args.no_save = True

#model_args.eval_batch_size = 4
model_args.eval_batch_size = 16
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.evaluate_during_training_steps = 50

model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.save_steps = -1

model_args.max_length = 64
#model_args.max_seq_length = 32
model_args.num_return_sequences = 10
#model_args.top_k = 50
model_args.top_k = 30
#model_args.top_p = 0.95
model_args.top_p = 0.97
model_args.fp16 = False
#model_args.num_beams = None

model_args.use_early_stopping = False
#model_args.early_stopping_delta = 0.01
#model_args.early_stopping_metric = "mcc"
#model_args.early_stopping_metric_minimize = False
#model_args.early_stopping_patience = 5
#model_args.evaluate_during_training_steps = 500
model_args.wandb_project =  "visualization-demo"

In [12]:
model = Seq2SeqModel(
    encoder_decoder_type="bart", encoder_decoder_name="outputs", args=model_args
)

print(
    model.predict(
        [
            "Non-GAAP gross profit was $225.4 million, representing an 86.7% non-GAAP gross margin, compared to $133.5 million, or a 85.3% non-GAAP gross margin, in the second quarter of fiscal 2022"
        ]
    )
)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[['Non-GAAP gross profit is $225.4 million and non-GAap gross margin is 86.7%.', 'Non-GAAP gross profit is $225.4 million and non-GAap gross margin is 86.7%.', 'Non-GAAP gross profit is $225.4 million and non-GAap gross margin is 86.7%.', 'Non-GAAP gross profit is $225.4 million.', 'Non-GAAP gross profit is $225.4 million.', 'Non-GAAP gross profit is $225.4 million.', 'Non-GAAP gross profit is $225.4 million and non-GAap gross margin is 86.7%.', 'Non-GAAP gross profit is $225.4 million.', 'Non-GAAP gross profit is $225.4 million and non-GAap gross margin is 86.7%.', 'Non-GAAP gross profit is $225.4 million and non-GAap gross margin is 86.7% in Q2.']]


In [10]:
createTrainingData(testDir, model)

Input file ../../Summary/DATA/PARAPHRASE/Test\ZS_2022-05-26_EP_YH.txt
Detected: 

Detected: 

Detected: 

Detected: 

Detected: 



Detected: 

Detected: 



Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: --
Detected: 


Detected: 

Detected: 



Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: Story
Detected: 



Detected: 


Detected: 


Detected: 


Detected: 


Detected: 



Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


Detected: 



Detected: 


Detected: 



Detected: 

Detected: 

Detected: 

Detected: 

Detected: 




Detected: 


Detected: 


Detected: 


Detected: 


Detected: 


71
ORIGINAL SENTENCE
  Zscaler, Inc.
PREDICTED PARAPHRASE



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[[' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.', ' Zscaler, Inc.']]


ORIGINAL SENTENCE
  Third Quarter Highlights.
PREDICTED PARAPHRASE



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[[' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.', ' Third Quarter Highlights.']]


ORIGINAL SENTENCE
  Revenue grows 63% year-over-year to $286.8 million.
PREDICTED PARAPHRASE



...

[[" Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022.", " Bank of America's 2022 Global Technology Conference Wednesday, June 8, 2022."]]


ORIGINAL SENTENCE
  4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.
PREDICTED PARAPHRASE



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[[' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.', ' 4th Annual Mizuho Cybersecurity Summit 2022 Monday, June 13, 2022.']]


ORIGINAL SENTENCE
  Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.
PREDICTED PARAPHRASE



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[[' Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.', ' Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.', ' Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.', ' Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.', ' Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.', ' Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.', ' Sessions which offer a webcast will be available on the Investor Relations section of the Zscaler website at https://ir.zscaler.com.', ' Sessions which offer a webcast will b

In [14]:
createTrainingData(sptrainDir, model, writetofile=True)
writeTrainingData(trainSPFile, sptrainDir)

Input file ../../Summary/DATA/SENTPAIR/Train\APPN_2022-11-03_EP_YH.txt
Input file APPN_2022-11-03_EP_YH
../../Summary/DATA/SENTPAIR/Train
SP file ../../Summary/DATA/SENTPAIR/Train/APPN_2022-11-03_EP_YH_sp.tsv
SP file ..\..\Summary\DATA\SENTPAIR\Train\APPN_2022-11-03_EP_YH_sp.tsv already exists
Input file ../../Summary/DATA/SENTPAIR/Train\BILL_2022-11-03_EP_YH.txt
Input file BILL_2022-11-03_EP_YH
../../Summary/DATA/SENTPAIR/Train
SP file ../../Summary/DATA/SENTPAIR/Train/BILL_2022-11-03_EP_YH_sp.tsv
SP file ..\..\Summary\DATA\SENTPAIR\Train\BILL_2022-11-03_EP_YH_sp.tsv already exists
Input file ../../Summary/DATA/SENTPAIR/Train\CRWD_2022-06-02_EP_YH.txt
Input file CRWD_2022-06-02_EP_YH
../../Summary/DATA/SENTPAIR/Train
SP file ../../Summary/DATA/SENTPAIR/Train/CRWD_2022-06-02_EP_YH_sp.tsv
SP file ..\..\Summary\DATA\SENTPAIR\Train\CRWD_2022-06-02_EP_YH_sp.tsv already exists
Input file ../../Summary/DATA/SENTPAIR/Train\CRWD_2023-05-31_EP_YH.txt
Input file CRWD_2023-05-31_EP_YH
../../Summa

In [15]:
createTrainingData(spdevDir, model, writetofile=True)
writeTrainingData(devSPFile, spdevDir)

Input file ../../Summary/DATA/SENTPAIR/Dev\BILL_2023-02-02_EP_YH.txt
Input file BILL_2023-02-02_EP_YH
../../Summary/DATA/SENTPAIR/Dev
SP file ../../Summary/DATA/SENTPAIR/Dev/BILL_2023-02-02_EP_YH_sp.tsv
SP file ..\..\Summary\DATA\SENTPAIR\Dev\BILL_2023-02-02_EP_YH_sp.tsv already exists
Input file ../../Summary/DATA/SENTPAIR/Dev\CFLT_2023-01-30_EP_YH.txt
Input file CFLT_2023-01-30_EP_YH
../../Summary/DATA/SENTPAIR/Dev
SP file ../../Summary/DATA/SENTPAIR/Dev/CFLT_2023-01-30_EP_YH_sp.tsv
SP file ..\..\Summary\DATA\SENTPAIR\Dev\CFLT_2023-01-30_EP_YH_sp.tsv already exists
Input file ../../Summary/DATA/SENTPAIR/Dev\CRWD_2022-11-29_EP_YH.txt
Input file CRWD_2022-11-29_EP_YH
../../Summary/DATA/SENTPAIR/Dev
SP file ../../Summary/DATA/SENTPAIR/Dev/CRWD_2022-11-29_EP_YH_sp.tsv
SP file ..\..\Summary\DATA\SENTPAIR\Dev\CRWD_2022-11-29_EP_YH_sp.tsv already exists
Input file ../../Summary/DATA/SENTPAIR/Dev\DDOG_2023-05-04_EP_YH.txt
Input file DDOG_2023-05-04_EP_YH
../../Summary/DATA/SENTPAIR/Dev
SP fi

In [11]:
train_df = pd.read_csv(trainSPFile, sep="\t", encoding = "utf-8").astype(str)
eval_df = pd.read_csv(devSPFile, sep="\t", encoding = "utf-8").astype(str)

train_df = train_df.rename(
    columns={"Sentence1": "text_a", "Sentence2": "text_b", "Target": "labels"}
)
eval_df = eval_df.rename(
    columns={"Sentence1": "text_a", "Sentence2": "text_b", "Target": "labels"}
)

train_df = train_df[["text_a", "text_b", "labels"]]
eval_df = eval_df[["text_a", "text_b", "labels"]]

#train_df["prefix"] = "sentpair"
#train_df = train_df[["Sentence1", "Sentence2", "Target"]]

#eval_df["prefix"] = "sentpair"
#eval_df = eval_df[["Sentence1", "Sentence2", "Target"]]

train_df = train_df.dropna()
train_df = train_df[train_df['text_a'].notna()]

eval_df = eval_df.dropna()
eval_df = eval_df[eval_df['text_a'].notna()]

train_df["text_a"] = train_df["text_a"].apply(clean_unnecessary_spaces)
train_df["text_b"] = train_df["text_b"].apply(clean_unnecessary_spaces)
train_df["labels"] = train_df["labels"].astype(int)
print("TRAIN DATA ..............")
print(train_df)

eval_df["text_a"] = eval_df["text_a"].apply(clean_unnecessary_spaces)
eval_df["text_b"] = eval_df["text_b"].apply(clean_unnecessary_spaces)
eval_df["labels"] = eval_df["labels"].astype(int)
print("EVAL DATA ..............")
print(eval_df)
#print(train_df["labels"].value_counts())

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


TRAIN DATA ..............
                                                 text_a  \
0                                   Appian Corporation.   
1      Third quarter cloud subscription revenue incr...   
2           MCLEAN, Va., Nov. 03, 2022 (GLOBE NEWSWIRE)   
3     -- Appian (Nasdaq: APPN) today announced finan...   
4             Third Quarter 2022 Financial Highlights:.   
...                                                 ...   
1142  Full fiscal year non-GAAP diluted EPS is expec...   
1143  Full fiscal year non-GAAP diluted EPS is expec...   
1144  Full fiscal year non-GAAP diluted EPS is expec...   
1145  Full fiscal year non-GAAP diluted EPS is expec...   
1146  Full fiscal year non-GAAP diluted EPS is expec...   

                                                 text_b  labels  
0                                   Appian Corporation.       1  
1      Third quarter cloud subscription revenue is $...       1  
2           MCLEAN, Va., Nov. 03, 2022 (GLOBE NEWSWIRE)       1  
3

In [13]:
spmodel_args = ClassificationArgs()
spmodel_args.do_sample = True
spmodel_args.train_batch_size = 8
spmodel_args.use_multiprocessing = False
spmodel_args.num_train_epochs = 5
#spmodel_args.learning_rate = 5e-5
#spmodel_args.no_save = True

spmodel_args.eval_batch_size = 8
spmodel_args.evaluate_generated_text = True
spmodel_args.evaluate_during_training = True
spmodel_args.evaluate_during_training_verbose = True
spmodel_args.evaluate_during_training_steps = 50

spmodel_args.overwrite_output_dir = True
spmodel_args.reprocess_input_data = True
spmodel_args.save_eval_checkpoints = False
spmodel_args.save_model_every_epoch = False
spmodel_args.save_steps = -1

spmodel_args.max_length = 64
#spmodel_args.max_seq_length = 512
#spmodel_args.num_return_sequences = 10
#spmodel_args.top_k = 50
#spmodel_args.top_p = 0.95
spmodel_args.fp16 = False
#spmodel_args.num_beams = None

spmodel_args.use_early_stopping = True
spmodel_args.early_stopping_delta = 0.01
spmodel_args.early_stopping_metric = "mcc"
spmodel_args.early_stopping_metric_minimize = False
spmodel_args.early_stopping_patience = 5
#spmodel_args.wandb_project =  "visualization-demo"

spmodel_args.output_dir = "spoutputs/"
#spmodel_args.lazy_loading = True

In [13]:
spmodel = ClassificationModel(
    "roberta", 
    "roberta-base",
    args=spmodel_args,
    use_cuda=True,
)

#spmodel = ClassificationModel(
#    "bert", 
#    "bert-base-uncased",
#    args=spmodel_args,
#    use_cuda=True,
#)

#model = T5Model("t5", "t5-base", args=model_args, use_cuda=True)

# Train the model
spmodel.train_model(
    train_df, eval_df=eval_df, acc=sklearn.metrics.accuracy_score
)
results = spmodel.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_2_3


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/144 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/342 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_3
INFO:simpletransformers.classification.classification_model:{'mcc': 0.677875562907472, 'tp': 219, 'tn': 77, 'fp': 29, 'fn': 17, 'auroc': 0.9243284298049249, 'auprc': 0.9612661118913268, 'acc': 0.8654970760233918, 'eval_loss': 0.566217239068951}
INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to spoutputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/342 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_3


Running Evaluation:   0%|          | 0/43 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.677875562907472, 'tp': 219, 'tn': 77, 'fp': 29, 'fn': 17, 'auroc': 0.9243284298049249, 'auprc': 0.9612661118913268, 'acc': 0.8654970760233918, 'eval_loss': 0.566217239068951}


In [14]:
spmodel = ClassificationModel(
    "roberta", "spoutputs", args=spmodel_args
)

predictions, raw_outputs = spmodel.predict(
    [
        [
            "Total revenue was $260.0 million, an increase of 66% year-over-year",
            "Total revenue is $260.0 million"
        ]
    ]
)
print(predictions)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[1]


In [18]:
# Get paraphrasing with ppmodel and spmodel
files = glob.glob(sptestDir+"/*_EP_YH.txt")
#print(files)
#print(inputDir)
if(len(files) > 0):
    for file in (files):
        print("Input file " + file)
        sentences = getSentences(file, nlp)
        for line in sentences:
            #print(line)
            nline = predictPhrase(line, model, spmodel)
            print(line, nline)

Input file ../../Summary/DATA/SENTPAIR/Test\BILL_2023-02-02_EP_YH.txt
59
https://finance.yahoo.com/news/bill-reports-second-quarter-fiscal-210600060.html https://finance.yahoo.com/news/bill-reports-second-quarter-fiscal-210600060.html
GAAP Net Loss Per Share is $(0.90) in second quarter 2023 @@@ GAAP Net Loss Per Share is $(0.90) in second quarter 2023 @@@
Cash And Cash Equivalents is $1616758 T in second quarter 2023 @@@ Cash And Cash Equivalents is $1616758 T in second quarter 2023 @@@
GAAP Gross Profit is $212539 T in second quarter 2023 @@@ GAAP Gross Profit is $212539 T in second quarter 2023 @@@
GAAP Gross Margin is 81.74% in second quarter 2023 @@@ GAAP Gross Margin is 81.74% in second quarter 2023 @@@
GAAP Free Cash Flow is $53.4MN second quarter 2023 @@@ GAAP Free Cash Flow is $53.4MN second quarter 2023 @@@
GAAP Free Cash Flow Margin is 20.56% in second quarter 2023 @@@ GAAP Free Cash Flow Margin is 20.56% in second quarter 2023 @@@


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Total Revenue in Q4 Fy23guidance is $619.1 MN - $628.2 million  . Total Revenue in Q4 is expected to be between $619.1 MN - $628.2 million .


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [15]:
# Create Para Phrasing training data using ppModel and spModel
ppDataDir = "../../Summary/Refined/PLTR/"
ppTrainDir = "../../Summary/DATA/PARAPHRASE/Train"
ppDevDir = "../../Summary/DATA/PARAPHRASE/Dev"
files = glob.glob(ppDataDir+"/*_EP_YH.txt")
#print(files)
#print(inputDir)
if(len(files) > 0):
    for file in (files):
        print("Input file " + file)
        basefile = os.path.basename(file)
        inputfile = os.path.splitext(basefile)[0]
        inputfile = inputfile+"_phrase.tsv"
        inputfilePath = ppTrainDir+"/"+inputfile
        inputfileDevPath = ppDevDir+"/"+inputfile
        print(inputfilePath, inputfileDevPath)
        outfile = Path(inputfilePath)
        outdevfile = Path(inputfileDevPath)
        if outfile.is_file():
            print(inputfilePath + " Already exists")
        elif outdevfile.is_file():
            print(inputfileDevPath + " Already exists")
        else:
            print("Creating training file {} for paraphrasing".format(inputfilePath))
            with open(outfile, "w", encoding = "utf-8") as of:
                of.write("filename\tSentence1\tSentence2\n")
                sentences = getSentences(file, nlp)
                for line in sentences:
                    #print(line)
                    pline, isProcess = preProcessSent(line)
                    if not isProcess:
                        continue
                    nline = predictPhrase(line, model, spmodel)
                    of.write(inputfile+"\t"+pline+"\t"+nline+"\n")
                    print(pline, nline)
        #break

Input file ../../Summary/Refined/PLTR\PLTR_2022-05-09_EP_YH.txt
../../Summary/DATA/PARAPHRASE/Train/PLTR_2022-05-09_EP_YH_phrase.tsv ../../Summary/DATA/PARAPHRASE/Dev/PLTR_2022-05-09_EP_YH_phrase.tsv
Creating training file ../../Summary/DATA/PARAPHRASE/Train/PLTR_2022-05-09_EP_YH_phrase.tsv for paraphrasing
43


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

 An investor presentation, including supplemental financial information and reconciliations of certain non-GAAP measures to their nearest comparable GAAP measures, will be available through Palantir’s Investor Relations website at https://investors.palantir.com.  An investor presentation, including supplemental financial information and reconciliations of certain non-GAAP measures to their nearest comparable GAAP measures, will be available through Palantir’s Investor Relations website at https://investors.palantir.com.
Input file ../../Summary/Refined/PLTR\PLTR_2023-05-08_EP_YH.txt
../../Summary/DATA/PARAPHRASE/Train/PLTR_2023-05-08_EP_YH_phrase.tsv ../../Summary/DATA/PARAPHRASE/Dev/PLTR_2023-05-08_EP_YH_phrase.tsv
../../Summary/DATA/PARAPHRASE/Train/PLTR_2023-05-08_EP_YH_phrase.tsv Already exists


In [26]:
# Translate proccessed ER text data to new paraphrase with the help of pmodel and spmodel
origFile = "../../Summary/Refined/APPN/APPN_2022-11-03_EP_YH.txt"
postRefDir = "../../Summary/PostRefined/"
splTag = ["SCHQ***", "SCBQ***", "SCBF***", "SCHF***", "SCG***", "GF***", "GQ***", "SC***"]

basefile = os.path.basename(origFile)
#inputfile = os.path.splitext(basefile)[0]
csym = basefile.split("_")[0]
#print(csym)
postRefDir = postRefDir + "/" + csym
if not os.path.exists(postRefDir):
    os.makedirs(postRefDir)
outfilePath = postRefDir + "/" + basefile
print(outfilePath)
outfile = Path(outfilePath)
if outfile.is_file():
    print("Post Refined Para Phrase file " + str(outfile) + " already exists")
else:
    with open(outfile, "w", encoding = "utf-8") as of:
        f = open(origFile, 'r', encoding="utf-8")
        line = f.readline()
        while line:
            if("ED***" in line):
                break
            elif("PG***" in line or "NOPAD***" in line):
                nline = None
                sentences = getSentences(None, nlp, line)
                for l in sentences:
                    #print(l)
                    pline, isProcess = preProcessSent(l)
                    if not isProcess:
                        continue
                    if not nline:
                        if("PG***" in line):
                            nline = "PG*** "
                        elif("NOPAD***" in line):
                            nline = "NOPAD*** "
                    pp = predictPhrase(l, model, spmodel)
                    if("Invalid Sentence" in pp):
                        continue
                    nline = nline + pp + " "
                if(nline):
                    nline = (nline[:len(nline)-1])
                    if(nline[len(nline)-1] == "."):
                        nline = nline[:len(nline)-1] + " ."
                    else:
                        nline = nline + " ."
                    print(nline)
                    of.write(nline+"\n\n")
            else:
                if(line == "\n"):
                    line = f.readline()
                    continue
                print(line)
                of.write(line+"\n\n")
            line = f.readline()
        f.close()

../../Summary/PostRefined//APPN/APPN_2022-11-03_EP_YH.txt
https://finance.yahoo.com/news/appian-announces-third-quarter-2022-200200184.html

GAAP Net Loss Per Share is $(0.61) in third quarter 2022 @@@

Cash And Cash Equivalents is $51802 T in third quarter 2022 @@@

GAAP Gross Profit is $84116 T in third quarter 2022 @@@

GAAP Gross Margin is 71.36% in third quarter 2022 @@@

ORIGINAL LINE:
 PG***


Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


PREDICTED PARAPHRASE:
 ['PG***']


  0%|          | 0/1 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


PREDICTED SENTENCE PAIR
 [['Non-GAAP net loss per share is expected to be between $(1.30) and $(1.36), assuming weighted average common shares outstanding of 72.5 million.', 'Non-GAAP net loss per share is expected to be between $(1.30) and $(1."36), assuming weighted average common shares outstanding of 72.5 million.']] [0]


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

PREDICTED SENTENCE PAIR
 [['Non-GAAP net loss per share is expected to be between $(1.30) and $(1.36), assuming weighted average common shares outstanding of 72.5 million.', 'Non-GAAP net loss per share is expected to be between $(1.30) and $(1.36), assuming weighted average common shares outstanding of 72.5 million.']] [1]
PG*** Non-GAAP net loss per share is expected to be between $(1.30) and $(1.36), assuming weighted average common shares outstanding of 72.5 million .


In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

model = Seq2SeqModel(
    encoder_decoder_type="bart", encoder_decoder_name="outputs"
)


while True:
    original = input("Enter text to paraphrase: ")
    to_predict = [original]

    preds = model.predict(to_predict)

    print("---------------------------------------------------------")
    print(original)

    print()
    print("Predictions >>>")
    for pred in preds[0]:
        print(pred)

    print("---------------------------------------------------------")
    print()

In [10]:
def load_data(file_path, input_text_column, target_text_column, label_column, keep_label=1):
    df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
    df = df.loc[df[label_column] == keep_label]
    df = df.rename(
        columns={input_text_column: "input_text", target_text_column: "target_text"}
    )
    df = df[["input_text", "target_text"]]
    df["prefix"] = "paraphrase"

    return df

In [6]:
train_df = pd.read_csv("../../Summary/ParaPhrase/final/train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("../../Summary/ParaPhrase/final/dev.tsv", sep="\t").astype(str)

train_df = train_df.loc[train_df["label"] == "1"]
eval_df = eval_df.loc[eval_df["label"] == "1"]

train_df = train_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
eval_df["prefix"] = "paraphrase"

In [7]:
train_df = train_df[["prefix", "input_text", "target_text"]]
eval_df = eval_df[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
eval_df = eval_df.dropna()
train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)

eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)

train_df = (train_df.loc[1:10000])
eval_df = (eval_df.loc[1:1000])
print(eval_df)

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


         prefix                                         input_text  \
1    paraphrase  They were there to enjoy us and they were ther...   
2    paraphrase  After the end of the war in June 1902, Higgins...   
3    paraphrase  From the merger of the Four Rivers Council and...   
4    paraphrase  The group toured extensively and became famous...   
5    paraphrase  Kathy and her husband Pete Beale ( Peter Dean ...   
..          ...                                                ...   
990  paraphrase  After his service Lockhart lived in Texas but ...   
991  paraphrase  After medical treatment, Strozzi started takin...   
992  paraphrase  In December 1969 became the 49th Army - Divisi...   
993  paraphrase  In `` The Stand '' by Glen Pequod Bateman, Woo...   
996  paraphrase  Dora Rangelova is the current captain of the B...   

                                           target_text  
1    They were there for us to enjoy and they were ...  
2    In August, after the end of the war in J

In [8]:
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"
model_args = Seq2SeqArgs()
model_args.do_sample = True
#model_args.eval_batch_size = 64
model_args.eval_batch_size = 32
model_args.evaluate_during_training = True
#model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_steps = 1000
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
#model_args.max_length = 128
model_args.max_length = 32
#model_args.max_seq_length = 128
model_args.max_seq_length = 32
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
#model_args.train_batch_size = 8
model_args.train_batch_size = 4
model_args.use_multiprocessing = False
#model_args.wandb_project = "Paraphrasing with BART"


model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-base",
    args=model_args,
)
model.train_model(train_df, eval_data=eval_df)

to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
]
truth = eval_df["target_text"].tolist()

preds = model.predict(to_predict)

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/4491 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model: Training started


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/1123 [00:00<?, ?it/s]

...

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.5267087161540985}
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/best_model
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-2246-epoch-2
INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/451 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.52150759100914}
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/best_model
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-base model complete. Saved to outputs/.


Generating outputs:   0%|          | 0/15 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 222.00 MiB (GPU 0; 4.00 GiB total capacity; 2.68 GiB already allocated; 0 bytes free; 3.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
print(preds)

In [24]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

In [25]:
sequence = "Non-GAAP net loss per share is expected to be between $(0.36) and $(0.42)"
tokens = tokenizer.tokenize(sequence)

print(tokens, len(tokens))

['Non', '-', 'GA', 'AP', 'Ġnet', 'Ġloss', 'Ġper', 'Ġshare', 'Ġis', 'Ġexpected', 'Ġto', 'Ġbe', 'Ġbetween', 'Ġ$(', '0', '.', '36', ')', 'Ġand', 'Ġ$(', '0', '.', '42', ')'] 24
